<a href="https://colab.research.google.com/github/annikaaross/Homochirality-project/blob/lio/minimal_functioning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homochirality Model

Hello and welcome to Do Opposites Attract?, the first monomer dating show. We're here to see if similarity between member(s) of a relationship matters in the strength of the bond between the individuals. Read on as we introduce you to our monomers, our polymers, and the stylish bag that they reside in. Along the way we will keep you up to date on all the drama of falling in love and breaking hearts.

## Global Parameters

Here's where we keep the rules by which our contestants must play.

In [1]:
#the bond break probbility applied to all bonds
BASE_BOND_BREAK_PROBABILITY = 0.9
#if a bond itself is homochiral the base bond fator is multiplied by
#this factor
HOMOCHIRAL_BREAK_FACTOR = 0.9
#this factor is incorporated for each additional neighboring homochiral bond
# becomes less and less influential as the bond gets further away
HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR = 0.3
#decreases the break probability based on the length of the polymer
LENGTH_FACTOR = 0.6



#limits the influence of length on breakprobabiltiy(the larger the less influence it has)
N=40

#saves the break probabilities of sequences of polymers
break_prob_lookup_table = {}

poisson_dict={}

#used in Poisson distribution
LAMBDA=6
#################################################################
#these are the same factors as before but with on for left and  #
#one for right homochiral bonds.                                #
#used in biaseastbondbreakability method which can be choosen to#
#be used over the regular eastbondbreakability method in the    #
#self.reset_break_probability() method                          #
#################################################################
HOMOCHIRAL_BREAK_FACTOR_LEFT = 0.0
HOMOCHIRAL_BREAK_FACTOR_RIGHT = 0.9
HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_LEFT = 0.3
HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_RIGHT = 0.3

N_RANDS = 1000


rands = []


#These are the liklihoods that a monomer will choose to poof or bond, respectively.
#Used in the Reactables class.
POOF_CHANCE = 0.3333
BOND_PROB = 0.3333

# Shorthands #
L = True
R = False

## Imports and Helpers

First we install a package called ```jdc``` which allows us to define classes across different cells using the syntax

```
%%add_to our_class
def our_function(self, our_variable):
  print our_variable
```

The documentation on this magic function can be found at https://alexhagen.github.io/jdc/






In [2]:
!pip install jdc

Here are the imports.

In [3]:
import random
import numpy as np
import copy
from google.colab import widgets
import matplotlib.pyplot as plt
import jdc
import more_itertools
import pandas as pd
import plotly.express as px
import time
import uuid

And here are some helper functions.

In [4]:
def make_pool(n):
  """
  Return a list of n new monomers.
  """
  monomers = []
  for n in range(n):
    monomers.append(Monomer())
  return monomers


def eAnd(*args): # From https://stackoverflow.com/q/2770434
  return [all(tuple) for tuple in zip(*args)]

def get_rand():
  """A function to provide uniform random numbers"""
  if len(rands)<= 10:
    randsarray = np.random.rand(N_RANDS)
    rands.extend(randsarray.tolist())
  return rands.pop()

def poissonequation(k):
  #checks if value has already been calulated
  if k not in poisson_dict:
    #otherwise calculates the values
    p = np.exp(-LAMBDA)*((LAMBDA**k)/(np.math.factorial(k)))
    #adds to dictionary
    poisson_dict[k] = 1- np.sqrt(p)
  return poisson_dict[k]


## The Monomer class

These are our Monomers. The individuals in our dating show. They are the best thing a girl can be in this worl, beautiful little fools. They know that they exist and they know what kind of monomer they are, left or right(thats handedness not political party alignment) What more can you ask for?

### Constructor

In [5]:
class Monomer:

  def __init__(self, **kwargs):
    """
    The constructor method for Monomer objects. Assigns default handedness.
    Accepts argument hand = bool with keyword.
    Considering other syntax for passing in handedness.
    Keeps track of how old the monomer is
    """
    import random
    self._handedness = kwargs['hand'] if 'hand' in kwargs else random.choice([True,False])
    self._eastbrkprob = kwargs['brkprob'] if 'brkprob' in kwargs else -1
    #keeps track of its age, the number of iterations it has survived
    self._age = 0
    self._uuid = uuid.uuid1()
  
  def __repr__(self):
    return str(f"{self.get_handedness()}-monomer")


### Getters and Setters

In [6]:
%%add_to Monomer

def get_handedness(self):
  """Getter method for Monomer handedness."""
  return self._handedness

def get_age(self):
  """Getter method for Monomer age"""
  return self._age


def get_eastbrkprob(self):
  """Getter method for east bond brk probability"""
  return self._eastbrkprob

def set_eastbrkprob(self, newbrk):
  """Setter method for east bond brk probability"""
  self._eastbrkprob = newbrk

def set_age(self, newage):
  """setter method for age"""
  self._age = newage

def reset_eastbrkprob(self) :
  """reset east brk probability to -1 
      to be called when single monomer is broken from polymer"""
  self.set_eastbrkprob(-1)

def get_id(self):
  return self._uuid

### Polymer Compatibility

In [7]:
%%add_to Monomer

def get_sequence(self):
  return (self.get_handedness(),)

def birthday(self):
  """ages the monomer up"""
  self._age += 1

## The Polymer class

This is our Polymer class, aka the relationships in this dating show. They contain a list of all the monomers that are inside of them. It is kept in a specific order. They are pretty open books. They can tell us how many monomers they contain, they add new members(either monomers or whole other polymers), they can remove entire groups of members after a certain index, they can check which of their bonds are homochiral and use this to calculate probability of a certain bond breaking(the weak areas in their relationship). They can use this to then tell the producers if and where they are planning on breaking off members of they ranks. Like all good relationships, they use purely mathematics and chance to determine these heart breaking decisions. They can also tell us how many left handed monomers and how many right handed monomers they contain. 

### Constructor

In [8]:
class Polymer:
  def __init__(self, monomers = []):
    """ Constructor method for Polymer.
    In addition to setting the monomers list, also calculates all break probabilities
    of the bonds it contains using reset_break_probability.
    """
    self._monomers = monomers
    self._age = 0
    self._uuid = uuid.uuid1()

  def __str__(self):
    """ str function for Polymer class. """
    return f"\nA polymer of length {self.get_length()}: {self.get_sequence()}"
  
  def __repr__(self):
    """ repr function for Polymer class. """
    return f"{self.get_sequence()}-polymer"

### Getters, Setters, and Logs

In [9]:
%%add_to Polymer


def get_monomers(self):
  """ Return list of monomers in the polymer. """
  return self._monomers


def get_length(self):
  """ Return the number of monomers in the polymer. """
  return len(self._monomers)


def get_sequence(self):
  """ Return the sequence of monomers in the polymer in a human-readable (and loggable) format. """
  sequence = []
  for monomer in self.get_monomers():
    sequence.append(monomer.get_handedness())
  return sequence

def get_age(self):
  """
  Getter method for the age of the polymer
  """
  return self._age


def set_age(self, newage):
  """
  Setter method for the age of the polymer
  """
  self._age = newage

def set_monomers(self, new_monomers):
  """ Set the monomers in the polymer to new_monomers. """
  self._monomers = new_monomers

def birthday(self):
  """
  Ages up the polymer and all of the monomers in the polymer
  """
  self._age += 1
  for n in range(self.get_length()):
    self.get_monomers()[n].birthday()
  
def get_id(self):
  return self._uuid

### Functionality methods

In [10]:
%%add_to Polymer

def append(self, other):
  """
  Adds a reactable (monomer or polymer) to the end of the Polymer.
  If the added reactable is a polymer, append() deletes it after taking its monomers.
  """
  #Check the type of thing to add
  if isinstance(other, Monomer):
    #Monomers can just be appended to the list
    self._monomers.append(other)
  elif isinstance(other, Polymer):
    #For polymers we need to extend rather than append, to avoid nested lists
    self._monomers.extend(other.get_monomers())
    #Then once the monomers are safely in their new polymer, the old one is deleted
    del other

def brkloc(self):

  #makes a list of all the indices of monomers in the polymer except 
  #for the last one
  indices = []
  for n in range(self.get_length() - 1):
    indices.append(n)
  #puts indices in random order
  random.shuffle(indices)
  #tests if each indices will break
  for index in indices:
    #gets the breakprobability for monomer at n index
    brkprob = self._monomers[index].get_eastbrkprob()
    rand = get_rand()
    # checks if the random number is less than the brkprob then the index of
    #the monomer is returned from the method
    #indicating this polymer breaks at the east bond of this monomer
    if(brkprob > rand):
      return index
  return None

def removeright(self,location):
  """
  Removes all monomers to the right (east) of a given index
  """
  #creates a list of monomers in the polymer to the left, and including,
  #the monomer at the location(index) provided
  newList = self._monomers[0:location+1]

  #makes a list of the monoers to the rightof the monomer at the 
  #index provided
  removed = self._monomers[location+1:]

  #sets the polymers monomers to newList(the monomers to the left
  #of  and the index provided)
  self.set_monomers(newList)
  #print("after breaking ractable is:",self)
  #print("after breaking removed is:",removed)

  #returns the monomers that were removed, everything the right
  #of the index provided
  return removed

### Homochirality handling

In [11]:
%%add_to Polymer
def easthomochiralcheck(self,numbermonomer):
  """
  Takes the index of a monomer within the Polymer and returns whether its east bond is homochiral.
  """
  #First check that the index isn't out of bounds 
  if (0 > numbermonomer or numbermonomer >= self.get_length()-1):
    return False
  #get the handedness of this monomer and its east neighbor, and return whether or not they're equal (bool)
  return self._monomers[numbermonomer].get_handedness() == self._monomers[numbermonomer+1].get_handedness()

def easthomochiralbiascheck (self, numbermonomer):
  """ this method can be used in place of the east homochiral check method
  returns false if not homochiral, 3 if homochiral left, 5 if homochiral right"""

  #First check that the index isn't out of bounds or if not homochiral
  if (0 > numbermonomer or numbermonomer >= self.get_length()-1 or not self.easthomochiralcheck(numbermonomer)):
    return False
  
  #otherwise check if they are homochiral left or right
  elif self._monomers[numbermonomer].get_handedness():
      #means left homochiral
      return 3
  elif not self._monomers[numbermonomer].get_handedness():
      #means right homochiral
      return 5
  

def eastbondbreakprobability(self,numbermonomer):
  """
  Takes the index of a monomer within the Polymer
  Returns the probability that the monomer's east bond will break
  returns -3 if the monomer has no east bond
  """
  ##############################
  #Override to return an equal break chance for every bond in the polymer
  #Simply comment out this line to get the regular function
  #return (0.3)
  ##############################

  #First check if the monomer is the last one in the polymer
  if (numbermonomer >= self.get_length()-1):
    #If so, it doesn't have an east bond to break, so the probability is 0(return -3)
    return -3
    
  #now we initialize brk probability (brk)
  brk = (BASE_BOND_BREAK_PROBABILITY) * (LENGTH_FACTOR**(self.get_length()/N))
  #check if the east bond is homochiral
  if (self.easthomochiralcheck(numbermonomer)):
    #if so multiply it by homochiral break factor (shrinks probability)
    brk *= HOMOCHIRAL_BREAK_FACTOR
    #goes through method which checks and calculates benefit of all homochiral neighbors
    brk = self.checkforhomochiralneighbors(numbermonomer,brk,HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR)
  #end def: returns break probability of monomers east bond
  return brk

def biaseastbondbreakprobability(self,numbermonomer):
  """
  ***left-right sensitivity****
  Takes the index of a monomer within the Polymer
  Returns the probability that the monomer's east bond will break
  returns -3 if the monomer has no east bond
  """
  ##############################
  #Override to return an equal break chance for every bond in the polymer
  #Simply comment out this line to get the regular function
  #return (0.3)
  ##############################
  
  #First check if the monomer is the last one in the polymer
  if (numbermonomer >= self.get_length()-1):
    #If so, it doesn't have an east bond to break, so the probability is 0 (returns -3)
    return -3
  
  #initialize the brk probability
  brk = (BASE_BOND_BREAK_PROBABILITY) * (LENGTH_FACTOR**(self.get_length()/N))

  #check if the east bond is homochiral left
  if (self.easthomochiralbiascheck(numbermonomer) == 3):
    brk *= HOMOCHIRAL_BREAK_FACTOR_LEFT

    #run through function that recalculates brk based on benfits of homochiral neighbors
    brk = self.checkforhomochiralneighbors(numbermonomer, brk, HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_LEFT)

  #otherwise checks if bond is homochiral right
  elif (self.easthomochiralbiascheck(numbermonomer) == 5):
    brk *= HOMOCHIRAL_BREAK_FACTOR_RIGHT

    #run through function that recalculates brk based on benfits of homochiral neighbors
    brk = self.checkforhomochiralneighbors(numbermonomer,brk, HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_RIGHT)
   
  return brk

def checkforhomochiralneighbors(self, numbermonomer, brk, neighborfactor):

  """
  helps all the versions of the eastbond break calculating functions
  takes in a brk probability a number monomer and the desired neighbor improvement factor
  and calculates how the neighbors a bond will increase the strength of the bond
  it then returns the new brk prob to the eastbond brk function
  """
  brk = brk
  j = numbermonomer + 1
  #going right to check for homochiral neighbors
  while(self.easthomochiralcheck(j)):
    #calculates decrease to brk prob for a neighbor of that distance for the bond
    brk *= 1 - (neighborfactor**abs(j - numbermonomer))
    j += 1

  #going left to check for homochiral neighbors
  j=numbermonomer-1
  while(self.easthomochiralcheck(j)):
    #calculates decrease to brk prob for a neighbor of that distance for the bond
    brk *= 1-(neighborfactor**abs(numbermonomer - j))
    j -= 1

  return brk


def poissonbreakprobability(self,numbermonomer):
  """
  Takes the index of a monomer within the Polymer
  Returns the probability that the monomer's east bond will break
  based on Poisson distribution
  returns -3 if the monomer has no east bond
  """

  #First check if the monomer is the last one in the polymer
  if (numbermonomer >= self.get_length()-1):
    #If so, it doesn't have an east bond to break, so the probability is 0(return -3)
    return -3

  #With that out of the way, we initialize brk to the base probability of breaking a bond
  brk = (BASE_BOND_BREAK_PROBABILITY) * (LENGTH_FACTOR**(self.get_length()/N))

  #keeps track of the number of the total consecutive homochiral bonds
  homochiralcount=0 

  #check if the east bond is homochiral
  if (self.easthomochiralcheck(numbermonomer)):
    #if so homochiral count increases by 1
    homochiralcount +=1
    #brk gets benefited by the poisson distribution of homochiral count 
    brk *= poissonequation(homochiralcount)

    #set j to monomer to right
    j = numbermonomer+1

    #going right to check for neighboring homochirality
    while(self.easthomochiralcheck(j)):
      #if homochiral increases count
      homochiralcount += 1
      #recalculates brk prob
      brk *= poissonequation(homochiralcount)

      #check next monomer
      j += 1

    #going left to check for neighboring homochirality
    j=numbermonomer-1
    homochiralcount = 1
    while(self.easthomochiralcheck(j)):
      homochiralcount += 1
      brk *= poissonequation(homochiralcount)
      j -= 1

  return brk


def reset_break_probability(self):
    
  """ 
  resets the break probabilities of the monomers in a 
  sequence. 1)checks lookup table to see if similar polymer values
  have been caluclated, otherwise calculates values and adds
  polymer to lookup table 2)sets values to monomers.
 
  """
  #store self as list of booleans to be added to lookup table
  polyseq = self.get_monomers().copy()
  for n in range(self.get_length()):
    polyseq[n] = polyseq[n].get_handedness()

  ##########################
  # LEFT/RIGHT SENSITIVITY #
  ##########################
  # All stored sequences start with True. If the one being requested doesn't, invert so that it does
  #comment out if right/left sensitivity is included
  if polyseq[0] == True:
    polyseq = [not mono for mono in polyseq]

  #make polyseq a tuple able to be added as an index of a dictionary  
  polyseq = tuple(polyseq)

  # If the sequence is already in the table (dict), retrieves its previously calculated break probabilities
  if polyseq in break_prob_lookup_table:
    the_probs = break_prob_lookup_table[polyseq]
    
  else: # Otherwise calculate and add to the lookup table
    # Calculate the break probabilities
    the_probs = self.calculatebrkprob()
    #and add it to the lookup table
    break_prob_lookup_table[polyseq] = the_probs

  #set monomers eastbond brk probs to correlated values stored in the_probs
  for n in range (self.get_length()):
    self._monomers[n].set_eastbrkprob(the_probs[n])

  #end def: polymer break probabilities recalculated and added
  #to lookup table (if not already there)

def calculatebrkprob(self):
    
  """
  generates and returns a list of break probabilities for the monomers at each index of
  a polymer. This method can be altered to calculate normally
  with left/right sensitivity or with a poisson distribution
 
  """

  #initialize brk_probs as an empty list
  brk_probs = []
  
  #checks if polymer is of length one -meaning it is lone monomer
  if self.get_length() == 1 :
    #if so adds -1 to list(brk prob value assigned to individual monomers)
    brk_probs.append(-1)
    
  else:
    #otherwise goes through all the indices of the polymer,calculates its brk probability, and appends it to brk_probs 
    for n in range(self.get_length()):

      #############################################################
      #LEFT RIGHT SENSITIVITY/POISSON EQUATIONS CHOICE.           #
      #pick one-comment out the rest.                             #
      #############################################################

      #this line if uncommented will have left/right sensitivity
      #brk_probs.append(self.biaseastbondbreakprobability(n))

      #this line if uncommented will calculate probabilities normally
      #brk_probs.append(self.eastbondbreakprobability(n))

      #this line if uncommented will calculate brk probabilities with poisson distribution
      brk_probs.append(self.poissonbreakprobability(n))

  #returns in order list of brk probs for the east bond of all the monomers in a polymer
  return brk_probs


## The Reactables class

This is the Reactables class aka the stylish little bag we keep our contestants in. As all good dating shows, we define our members by the relationship they are in. So a single monomer will be addd to the bag alone, but a monomer in a polymer will not be. Rather each polymer will be recorded once. Its like if you have charlie, ben, and jack. Jack and Ben are in a relationship so they are jointly called Jen. If these three were in the reactable bag they would be recognized as two things, charlie and Jen(Ben+JAck). The reactable class is the heart and center of our game- I mean dating- show. Its here we ask all the members just the right questions to get them falling in love and breaking hearts, and then we are curteous enough to do all the splitting and merging for them.

### Constructor

In [12]:
class Reactables:

  def __init__(self, reactables = []):
    """
    Constructor class for Reactables object
    Accepts list of reactables or defaults to empty list
    Reactables functions as a bin for reactable objects monomer and polymer
    Methods consist mostly of list handling
    """
    self._reactables = reactables
    #The reactables bag also handles remembering its history
    self._history = []
    self._lookup = {}
    self._hist_stats = pd.DataFrame()


  def __str__(self):
    """
    str method for Reactables class
    """
    return str(self.get_reactables())

### Getters and Setters

In [13]:
%%add_to Reactables

#################
#GETTERS/SETTERS#
#################

def get_reactables(self):
  """
  Getter method for reactables in Reactables
  Returns a list of the objects currently in the Reactables bag
  """
  return self._reactables

def get_history(self):
  """ Return the history of the reactables bag.
  """
  return self._history

def set_history(self, new_hist):
  """ Set the history to now_hist. """
  self._history = new_hist
  
def get_count(self):
  """
  Getter method for Reactables class
  Returns the number of objects in the Reactables bag
  """
  return len(self.get_reactables()) 


def set_reactables(self, new_list):
  """
  Setter method for the Reactables class
  Sets the reactables bag to a new list
  Used mostly as a helper function in bonding
  """
  self._reactables = new_list

def get_stats(self):
  return self._hist_stats

def set_hist_stats(self,df):
  self._hist_stats = df

def get_lookup(self):
  return self._lookup

def add_stat_lookup(self, key, value):
  self._lookup[key] = value

def add_log(self, log):
  (self._history).append(log.copy())


### Functionality methods

#### Top level functionality

In [14]:
%%add_to Reactables

def refill(self, pool_size):
  #Do we have less than the required number of things in the pool?
  dearth = pool_size - self.get_count()
  #If so,
  if dearth > 0:
    new = []
    #Make as many monomers as we need to get back up to the riquired amount
    for n in range(dearth):
      new.append(Monomer())
    #And add them to the reactables bag
    self.add(new)

def randomize_reactables(self):
  """
  Randomizes the order of the reactables list
  """
  #Get the current reactables
  reactables = self.get_reactables()
  #Shuffle them
  random.shuffle(reactables)
  #And reset the reactables list to the new sorted list
  self.set_reactables(reactables)

def do_the_thing(self):
  """
  Handle a single iteration of the reactables.

  """
  #We keep track of bonding by storing the most recent reactable to choose to bond as the 'bachelor.'
  bachelor = None
  #We need a copy of the reactables because indices will change as soon as we start doing stuff
  reactables = copy.copy(self.get_reactables())
  #Iterate through each reactable
  for item in reactables:
    #Roll a random number in (0,1) to compare against our probabilities
    roll = get_rand()
    #If our reactable is a monomer...
    if isinstance(item, Monomer):
      #We get our breaking and bonding chances (parameters now, may become functions later?)
      brk = POOF_CHANCE
      bond = BOND_PROB
      #If we roll to break the monomer
      if 0 < roll <= brk:
        #Just delete it
        del item
      #If we roll to bond the monomer
      elif brk < roll <= brk+bond:
        #Check if there's a reactable waiting to bond
        if bachelor == None:
          #If not, make this monomer the bachelor. Someone will come along to bond later.
          bachelor = item
        else:
          #If there is someone waiting, bond them together and reset the bachelor chair to empty.
          self.bond_pair(bachelor, item)
          bachelor= None
      else:
        continue
    #If the reactable is a polymer instead...
    elif isinstance(item, Polymer):
      #Choose whether the polymer will break or bond. It's 50-50 right now.
      if roll >= 0.33:
        #This is the same bonding logic as for the monomer. It's not in a helper function because it needs to be able to access bachelor.
        if bachelor == None:
          bachelor = item
        else:
          self.bond_pair(bachelor, item)
          bachelor = None
      #Here's what happens if the polymer is chosen to check breaking
      else:
        #It finds its break location
        break_spot = item.brkloc()
        #Which might turn out to be nowhere.
        if break_spot != None:
          #If somewhere does break, call break_polymer to handle the breaking
          self.break_polymer(item,break_spot)
    else:
      #You never know what might end up in your reactables bag
      raise ValueError("This thing is neither a monomer or a polymer. What?!")

def iterate(self,size,iteration_number):
  """
  Handles full sequence of iteration
  """
  self.randomize_reactables()
  self.do_the_thing()
  self.log(iteration_number)
  self.ageup()
  self.refill(size)

def simulate(self, poolsize, iterations):
  for n in range(iterations):
    self.iterate(poolsize, n)
  self.parse_history()


#### Functionality helpers

In [15]:
%%add_to Reactables


def add(self, new_reactables):
  """
  Adds a list of reactables to the reactables bag
  Built on extend(); argument must be iterable
  """
  #print("list of new Reactbles", new_reactables)
  #print("list of get Reactables:",self.get_reactables())
  self.get_reactables().extend(new_reactables)
  #print("reactables list is now",self.get_reactables())


def subtract(self, removables):
  """
  Removes a list of reactables from the reactables bag
  Built on list comprehension; argument must be iterable
  """
  self.set_reactables([reactable for reactable in self.get_reactables() if not (reactable in removables)])


def bond_pair(self, west, east):
  """
  Accepts two reactables from the bag
  Bonds them together, handling deletion of any emptied polymer
  """

  #this is my attempt to get the ages working properly the idea is:

  #if both are polymers or both are monomers:
  if (type(west) == type(east) and type(west) == Polymer):
    #set the new age to the maximum age of the polymers
    age = max(west.get_age(), east.get_age())
  
  if (type(west) == type(east) and type(west) == Monomer):
    #set age to 0 it is a brand new polymer
    age = 0

  #if only one is a polymer:
  elif isinstance(west, Polymer) and isinstance(east, Monomer):
    #set the age to the age of the polymer
    age = west.get_age()
  elif isinstance(west, Monomer) and isinstance(east, Polymer):
    # set the age to the age of the polymer
    age = east.get_age()

  #Take your two reactables and check if the west one is a monomer or a polymer 
  if isinstance(west, Polymer):
    #If it's a polymer, all you've got to do is append the east item and then take it out of the bag
    west.append(east)
    self.subtract([east])
    #recalculate the breakprobabilities of the bonds in polymer
    west.reset_break_probability()
    #set new polymer to age that was calculated before
    west.set_age(age)

  if isinstance(west, Monomer):
    #If west friend is a Monomer, then make a new polymer containing west
    newpoly = Polymer([west])
    #Add the east things to it
    newpoly.append(east)
    #Remove both the west monomer and the east thing
    self.subtract([east])
    self.subtract([west])
    #And put the new polymer into the reactables bag
    self.add([newpoly])
    #recalculates break probability
    newpoly.reset_break_probability()
    #set new polymer to age that was calculated before
    newpoly.set_age(age)



def break_polymer(self, polymer, brk_location):
  #breaks polymer at given location and creates a new polymer of the 
  #monomers removed
  #when polymer is made-breakprobabilities are calculated
  newPolymer = Polymer(polymer.removeright(brk_location))
  #if(newPolymer._monomers[0].get_handedness() and polymer._monomers[-1].get_handedness()):
    #print ("left breaking when literally impossible????")
  #if(not newPolymer._monomers[0].get_handedness() and not polymer._monomers[-1].get_handedness()):
   # print("right breaking")
  polys = [newPolymer, polymer]
  #resets break probabilities in polymers
  #goes throught the two new polymers
  for poly in polys:
    #resets break probabilities in polymers
    polymer.reset_break_probability()
    if poly.get_length()==0:
      print("whyyyyyyyyyyy is this happening????")
    #checks if they are of length 1
    if poly.get_length() <= 1:
      #adds them to reactable as a single monomer
      self.add(poly.get_monomers())
      #subtracts polymer from reactable list
      self.subtract([poly])
      del poly
    #checks if the polymer is not in the reactables list
    elif (poly not in self.get_reactables()):
      #add polymer to reactable list
      self.add([poly])

    



### Current state data methods

In [16]:
%%add_to Reactables

def ageup(self):
  """
  Method that ages up every reactable in the reactables bag
  """
  for reactable in self.get_reactables():
    reactable.birthday()
    




### History handling methods

In [17]:
%%add_to Reactables

def log(self, iteration):
  """ Translate current reactables into a loggable list of tuples.
  This log keeps the full sequences of the polymers without digesting the data, 
  so it should be more flexible as we more forward with pulling new information
  out of this simulation. 
  There is one major piece of information lost, however, which is the identities
  of the individual polymers and monomers. """

  # Column labels for History Handling #
  Type = "Type"                   # Monomer or polymer
  Length = "Length"               # How many monomers in the thing
  nLefts = "#Lefts"               # How many left monomers in the thing
  nRights = "#Rights"             # How many right monomers in the thing
  Sequence = "Sequence"           # The string sequence of the thing
  nLhomo = "#LeftHomochiral"      # The number of left homochiral bonds in the polymer
  nRhomo = "#RightHomochiral"     # The number of right homochiral bonds in the polymer
  nHomo = "#Homochiral"           # The overall number of homochiral bonds in the polymer
  sEE = "Signed ee"               # The signed enantiomeric excess of the polymer (+ if more True)
  pcHomo = "%Homochirality"       # The proportion of bonds in the polymer that are homochiral
  pcLhomo = "%LeftHomochirality"  # The proportion of bonds in the polymer that are left homochiral
  pcRhomo = "%RightHomochirality" # The proportion of bonds in the polymer that are right homochiral
  Iter = "Iteration"              # The iteration number at which the item is found
  Age = "Age"                     # The age of the item
  Id = "ID"                       # The ID of the item
  C_lens = "Chain lengths"        # A list of the chain lengths in the item
  Max_C = "Longest chain length"  # The length of the longest chain in the item


  for r in self.get_reactables():
    item = r.get_sequence()
    age = r.get_age()
    # Translate
    sequence = self.standard_form(item)

    #Is it in the lookup table?
    if sequence not in self.get_lookup():

      # If not, check what kind of data to gether
      if len(item) == 1: # It's a monomer
        # Get the info
        lr = self.count_LR(item)
        # And put it in the lookup table
        self.add_stat_lookup(sequence, {Type: 'Monomer', 
                                        Length:1, 
                                        nLefts:lr[0], 
                                        nRights:lr[1],
                                        Sequence:sequence})
      elif len(item) > 1: # It's a polymer
        # Get basic info
        length = len(item)
        lr = self.count_LR(item)
        bonds = self.homochiral_bond_counts(item)
        total_homos = bonds[0]+bonds[1]
        signed_ee = (lr[0]-lr[1])/(lr[0]+lr[1])
        homochirality = total_homos/(length-1)
        lhomochirality = bonds[0]/(length-1)
        rhomochirality = bonds[1]/(length-1)
        chains = self.chain_lengths(item)
        max_c = max(chains)
        # And put it in the lookup table
        self.add_stat_lookup(sequence, {Type: 'Polymer', 
                                        Length: length, 
                                        nLefts:lr[0], 
                                        nRights:lr[1],
                                        nLhomo:bonds[0],
                                        nRhomo:bonds[1],
                                        nHomo:total_homos,
                                        sEE:signed_ee,
                                        pcHomo:homochirality,
                                        pcLhomo:lhomochirality,
                                        pcRhomo:rhomochirality,
                                        Sequence:sequence,
                                        C_lens:chains,
                                        Max_C:max_c})
      else:
        raise ValueError("There's something with length 0 in your history.")

    # Now that the data is searchable...
    # Log it
    new_log = self.get_lookup().get(sequence)
    new_log[Iter] = iteration
    new_log[Age] = age
    new_log[Id] = r.get_id()
    self.add_log(new_log)


def parse_history(self):
  """ Create an array of plottable information from the history log. """
  parsed_hist = pd.DataFrame(self.get_history())
  self.set_hist_stats(parsed_hist)


#### Low level info functions

In [18]:
%%add_to Reactables

def count_LR(self, log):
    """ Return (n_Lefts, n_Rights) """
    if L and not R:
        return (sum(log), len(log)-sum(log))
    elif R and not L:
        return (len(log)-sum(log), sum(log))
    else:
        raise ValueError("Your L's and R's are screwed up somehow.")
  

def homochirality_sequence(self, log):
  """ Return a boolean list of bonds within a given logged polymer, True if the bond is
      homochiral. """
  return [i[0]==i[1] for i in list(more_itertools.pairwise(log))]


def homochiral_bond_counts(self,log):
    """ Return (number of left homochiral bonds, number of right homochiral bonds) """
    homo = self.homochirality_sequence(log)
    west_true = log[:-1]
    true_homochiral = eAnd(homo,west_true)
    west_false = [not m for m in west_true]
    false_homochiral = eAnd(homo,west_false)
    if L and not R:
        return (sum(true_homochiral), sum(false_homochiral))
    elif R and not L: 
        return (sum(false_homochiral), sum(true_homochiral))
    else:
        raise ValueError("Your L's and R's are screwed up somehow.")


def chain_lengths(self,polylog):
  """ Return the lengths of the homochiral chains in a given polymer log. """
  count = 1
  lengths = [] 
  for n in range(1,len(polylog)):
    if polylog[n] == polylog[n-1]:
      count += 1
    else:
      lengths.append(count)
      count = 1
  lengths.append(count)
  return lengths
    

def standard_form(self,poly):
  """ THIS NEEDS TO BE UPDATED TO BE FLEXIBLE FOR L AND R BOOLS """
  if len(poly) == 0:
    return ''
  return ''.join(['L' if m else 'R' for m in poly])
    


### Plotting methods

## Test program

In [19]:
POOL_SIZE = 10
ITERATIONS = 10

a = Reactables(make_pool(POOL_SIZE))
a.simulate(POOL_SIZE, ITERATIONS)

stats = a.get_stats()
print("Done.")

Done.
